<a href="https://colab.research.google.com/github/tankTopTaro/Deep-Learning-with-Python/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook will cover some fundamental concepts of tensors using TensorFlow

Thing to covers:
* Introduction to tensors
* Getting informations from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try for yourself!


# Introduction to Tensors

In [12]:
# Import TensorFlow
import tensorflow as tf
import numpy as np



print(tf.__version__)

2.8.2


In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions of a tensor
# (ndim stands for number of dimensions)
scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# Check the dimension of vector
vector.ndim

1

In [ ]:
# Create a matrix
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [ ]:
# Check the dimension of matrix
matrix.ndim

2

In [ ]:
# Create another matrix
# Specify the data type with dtype parameter
another_matrix = tf.constant([[10., 7.],
                             [3., 2.],
                             [8., 9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
# Check the dimension of another_matrix
another_matrix.ndim

2

In [ ]:
# Create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [ ]:
# Check the dimension of the tensor
tensor.ndim

3

* Scalar: a single number
* Vector: a number with direction
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (a 0-D tensor is a scalar, a 1-D tensor is a vector

### Creating tensors with `tf.Variable`

In [ ]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [ ]:
# Create the same tensors with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Try to change an element in changeable_tensor and unchangeable_tensor
changeable_tensor[0] = 7
changeable_tensor


TypeError: ignored

In [ ]:
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [ ]:
unchangeable_tensor[0] = 7
unchangeable_tensor

TypeError: ignored

In [ ]:
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: ignored

🔑 **Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors
Random tensors are tensors of some arbitrary size which contain random numbers.

In [ ]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7)    # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor
Shuffling is valuable for when you want to shuffle data so the inherent order doesn't affect learning

In [ ]:
# Create a non-shuffled tensor
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])

# Shuffle the non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

⚒ **Exercise:** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors

In [ ]:
# if global seed nor the operation seed is set, we get different results for 
## every call to the random op and every re-run of the program
print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'

tf.Tensor([0.6645621], shape=(1,), dtype=float32)
tf.Tensor([0.68789124], shape=(1,), dtype=float32)


In [ ]:
# if global seed is set but the operation seed is not set, we get different results for 
## every call to the random op, but the same sequence for every re-run of the program
tf.random.set_seed(1234)  # Global Seed
print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'
print(tf.random.uniform([1]))  # generates 'A3'

tf.Tensor([0.5380393], shape=(1,), dtype=float32)
tf.Tensor([0.3253647], shape=(1,), dtype=float32)
tf.Tensor([0.59750986], shape=(1,), dtype=float32)


In [ ]:
# if the operation seed is set, we get different results for every call to the random op,
## but the same sequence for every re-run of the program
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'
print(tf.random.uniform([1], seed=1))  # generates 'A3'

tf.Tensor([0.1689806], shape=(1,), dtype=float32)
tf.Tensor([0.7539084], shape=(1,), dtype=float32)
tf.Tensor([0.4243431], shape=(1,), dtype=float32)


In [ ]:
# global and operation seed is set
unshuffled = tf.constant([[[1, 2, 3],
                           [4, 5, 6]],
                          [[7, 8, 9],
                           [10, 11, 12]],
                          [[13, 14, 15],
                           [16, 17, 18]]])
tf.random.set_seed(20)
tf.random.shuffle(unshuffled, seed=20)

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[13, 14, 15],
        [16, 17, 18]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[ 1,  2,  3],
        [ 4,  5,  6]]], dtype=int32)>

In [ ]:
# no global seed nor operation seed is set
unshuffled = tf.constant([[[1, 2, 3],
                           [4, 5, 6]],
                         [[7, 8, 9],
                           [10, 11, 12]]])
tf.random.shuffle(unshuffled)

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]]], dtype=int32)>

In [ ]:
# operation seed is set
unshuffled = tf.constant([[[1, 2, 3],
                           [4, 5, 6]],
                         [[7, 8, 9],
                           [10, 11, 12]]])
tf.random.shuffle(unshuffled, seed=123)

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 7,  8,  9],
        [10, 11, 12]],

       [[ 1,  2,  3],
        [ 4,  5,  6]]], dtype=int32)>

In [ ]:
# global seed is set
unshuffled = tf.constant([[[1, 2, 3],
                           [4, 5, 6]],
                         [[7, 8, 9],
                           [10, 11, 12]]])
tf.random.set_seed(123)
tf.random.shuffle(unshuffled)

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 7,  8,  9],
        [10, 11, 12]],

       [[ 1,  2,  3],
        [ 4,  5,  6]]], dtype=int32)>

### Other ways to make tensors

In [ ]:
tf.ones([2, 4])

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [ ]:
tf.zeros(shape=(4, 2))

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)>

### Turn Numpy arrays into tensors

In [2]:
numpy_A = np.arange(1, 25, dtype=np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [ ]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting information from tensors
Tensor Attributes
* Shape — `tensor.shape`
* Rank — `tensor.ndim`
* Axis or dimension — `tensor[0], tensor[:, 1]...`
* Size — `tf.size(tensor)`


In [ ]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
# Axis
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [ ]:
# Shape, Rank, Size
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
# Visualize the attributes
print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimension (rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimension (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor:  120


### Indexing tensors
Tensors can be indexed just liked Python lists.

In [ ]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [ ]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
some_list[:1]

[1]

In [ ]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[4, 5],
                             [2, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [ ]:
some_list, some_list[-1]

([1, 2, 3, 4], 4)

In [ ]:
# Get the last item of each row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([5, 4], dtype=int32)>

In [ ]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[4],
        [5]],

       [[2],
        [4]]], dtype=int32)>

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1)    # '-1' means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[4],
        [5]],

       [[2],
        [4]]], dtype=int32)>

In [ ]:
tf.expand_dims(rank_2_tensor, axis=0)   # expand the 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[4, 5],
        [2, 4]]], dtype=int32)>

### Manipulating tensors (tensor operations)
**Basic operations**

`+`,`-`,`*`,`/`

In [ ]:
# Tensor addition
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
# Multiplication
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
# Subtraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [ ]:
# Division
tensor / 10

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

**Using tensorflow built-in function**

In [ ]:
# Addition
tf.add(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
# Multiplication
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
# Subtraction
tf.subtract(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [ ]:
# Division
tf.divide(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

### Matrix multiplication

**Rules when multipying matrices:**


1.   The inner dimensions must match.
2.   The resulting matrix has the shape of the outer dimension.



In [ ]:
# Matrix multiplication with tensorflow
tensor_1 = tf.constant([[1, 2, 5], [7, 2, 1], [3, 3, 3]])
tensor_2 = tf.constant([[3, 5], [6, 7], [1, 8]])
tf.matmul(tensor_1, tensor_2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [ ]:
# Matrix mulitplication with Python operator '@'
tensor_1 @ tensor_2

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [ ]:
tensor_1.shape, tensor_2.shape

(TensorShape([3, 3]), TensorShape([3, 2]))

In [ ]:
# Create tensors of same shape (3, 2)
X = tf.constant([[1, 2], [3, 4], [5, 6]])
Y = tf.constant([[7, 8], [9, 10], [11, 12]])
X.shape, Y.shape

(TensorShape([3, 2]), TensorShape([3, 2]))

In [ ]:
# Try to matrix multiply tensors of same shape via Python
X @ Y

InvalidArgumentError: ignored

In [ ]:
# Try to matrix multiply tensors of same shape via tensorflow
tf.matmul(X, Y)

InvalidArgumentError: ignored

📖 How to multiply matrices  https://www.mathsisfun.com/algebra/matrix-multiplying.html

In [ ]:
# Reshape Y
tf.reshape(Y, shape=(2, 3)), Y.shape

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[ 7,  8,  9],
        [10, 11, 12]], dtype=int32)>, TensorShape([3, 2]))

In [ ]:
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [ ]:
# Difference between transpose and reshape
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [ ]:
# Matrix Multiplication with transpose
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

**The dot product**

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`


In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Check the values of Y, reshaped Y, and transposed Y
print("Normal Y: ")
print(Y, "\n")

print("Y reshaped to shape=(2, 3): ")
print(tf.reshape(Y, shape=(2, 3)), "\n")

print("Y transposed: ")
print(tf.transpose(Y))

Normal Y: 
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to shape=(2, 3): 
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed: 
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


In [ ]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

### Changing the datatype of a tensor

In [3]:
# Create a tensor with default datatype (float32)
B = tf.constant([1.3, 4.5])
B.dtype

tf.float32

In [4]:
# Create a tensor with int32 datatype
C = tf.constant([1, 2])
C.dtype

tf.int32

In [5]:
# Change datatype from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.3, 4.5], dtype=float16)>,
 tf.float16)

In [6]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [7]:
E_float16 = tf.cast(E, dtype=tf.float16)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>,
 tf.float32)

In [9]:
# Change from float16 to int32
D_int32 = tf.cast(D, dtype=tf.int32)
D_int32, D_int32.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 4], dtype=int32)>,
 tf.int32)

In [11]:
# Change from float16 to int16
D_int16 = tf.cast(D, dtype=tf.int16)
D_int16, D_int16.dtype

(<tf.Tensor: shape=(2,), dtype=int16, numpy=array([1, 4], dtype=int16)>,
 tf.int16)

### Aggregating tensors

— condensing from multiple values down to a smaller amount of values

In [13]:
# Get the absolute values
D = tf.constant([-1, -2])
D, tf.abs(D)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-1, -2], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>)

Forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [24]:
# Mean & Sum
array_1 = np.random.randint(0, 100, size=(2, 2, 3))
tensor = tf.constant(array_1)

sum = tf.math.reduce_sum(tensor)
mean = tf.math.reduce_mean(tensor)

print("tensor: ", tensor)
print("Sum: ", sum)
print("Mean: ", mean)

tensor:  tf.Tensor(
[[[39 39 38]
  [50 32 98]]

 [[63 23 42]
  [15 84 86]]], shape=(2, 2, 3), dtype=int64)
Sum:  tf.Tensor(609, shape=(), dtype=int64)
Mean:  tf.Tensor(50, shape=(), dtype=int64)


In [25]:
# Min & Max
min = tf.math.reduce_min(tensor)
max = tf.math.reduce_max(tensor)

print("tensor: ", tensor)
print("min: ", min)
print("max: ", max)

tensor:  tf.Tensor(
[[[39 39 38]
  [50 32 98]]

 [[63 23 42]
  [15 84 86]]], shape=(2, 2, 3), dtype=int64)
min:  tf.Tensor(15, shape=(), dtype=int64)
max:  tf.Tensor(98, shape=(), dtype=int64)


⚒ **Exercise:** Find the variance and standard deviation of tensor using Tensorflow methods

📖 [Standard Deviation and Variance](https://www.mathsisfun.com/data/standard-deviation.html) 

In [27]:
# variance
E = tf.cast(tensor, dtype=tf.float32)
variance = tf.math.reduce_variance(E)

print("tensor: ", E)
print("variance: ", variance)

tensor:  tf.Tensor(
[[[39. 39. 38.]
  [50. 32. 98.]]

 [[63. 23. 42.]
  [15. 84. 86.]]], shape=(2, 2, 3), dtype=float32)
variance:  tf.Tensor(637.1875, shape=(), dtype=float32)


In [40]:
variance_axis_0 = tf.math.reduce_variance(E, 0)
print("tensor E: ", E)
print("variance_axis_0: ", variance_axis_0)

tensor E:  tf.Tensor(
[[[39. 39. 38.]
  [50. 32. 98.]]

 [[63. 23. 42.]
  [15. 84. 86.]]], shape=(2, 2, 3), dtype=float32)
variance_axis_0:  tf.Tensor(
[[144.    64.     4.  ]
 [306.25 676.    36.  ]], shape=(2, 3), dtype=float32)


In [41]:
variance_axis_1 = tf.math.reduce_variance(E, 1)
print("tensor E: ", E)
print("variance_axis_0: ", variance_axis_1)

tensor E:  tf.Tensor(
[[[39. 39. 38.]
  [50. 32. 98.]]

 [[63. 23. 42.]
  [15. 84. 86.]]], shape=(2, 2, 3), dtype=float32)
variance_axis_0:  tf.Tensor(
[[ 30.25  12.25 900.  ]
 [576.   930.25 484.  ]], shape=(2, 3), dtype=float32)


In [36]:
E.ndim

3

In [42]:
variance_axis_2 = tf.math.reduce_variance(E, 2)
print("tensor E: ", E)
print("variance_axis_2: ", variance_axis_2)

tensor E:  tf.Tensor(
[[[39. 39. 38.]
  [50. 32. 98.]]

 [[63. 23. 42.]
  [15. 84. 86.]]], shape=(2, 2, 3), dtype=float32)
variance_axis_2L  tf.Tensor(
[[2.2222222e-01 7.7600000e+02]
 [2.6688889e+02 1.0895555e+03]], shape=(2, 2), dtype=float32)


In [43]:
# Deliberate error
variance_axis_3 = tf.math.reduce_variance(E, 3)
print("tensor E: ", E)
print("variance_axis_3: ", variance_axis_3)

InvalidArgumentError: ignored

In [49]:
# standard deviation
std = tf.math.reduce_std(E)

print("tensor E: ", E)
print("\nstd: ", std)

tensor E:  tf.Tensor(
[[[39. 39. 38.]
  [50. 32. 98.]]

 [[63. 23. 42.]
  [15. 84. 86.]]], shape=(2, 2, 3), dtype=float32)

std:  tf.Tensor(25.242573, shape=(), dtype=float32)


In [48]:
std_axis_0 = tf.math.reduce_std(E, 0)

print("tensor E: ", E)
print("\nstd_axis_0: ", std_axis_0)

tensor E:  tf.Tensor(
[[[39. 39. 38.]
  [50. 32. 98.]]

 [[63. 23. 42.]
  [15. 84. 86.]]], shape=(2, 2, 3), dtype=float32)

std_axis_0:  tf.Tensor(
[[12.   8.   2. ]
 [17.5 26.   6. ]], shape=(2, 3), dtype=float32)


In [50]:
std_axis_1 = tf.math.reduce_std(E, 1)

print("tensor E: ", E)
print("\nstd_axis_0: ", std_axis_1)

tensor E:  tf.Tensor(
[[[39. 39. 38.]
  [50. 32. 98.]]

 [[63. 23. 42.]
  [15. 84. 86.]]], shape=(2, 2, 3), dtype=float32)

std_axis_0:  tf.Tensor(
[[ 5.5  3.5 30. ]
 [24.  30.5 22. ]], shape=(2, 3), dtype=float32)


In [51]:
std_axis_2 = tf.math.reduce_std(E, 2)

print("tensor E: ", E)
print("\nstd_axis_0: ", std_axis_2)

tensor E:  tf.Tensor(
[[[39. 39. 38.]
  [50. 32. 98.]]

 [[63. 23. 42.]
  [15. 84. 86.]]], shape=(2, 2, 3), dtype=float32)

std_axis_0:  tf.Tensor(
[[ 0.47140452 27.856777  ]
 [16.336735   33.008415  ]], shape=(2, 2), dtype=float32)


In [52]:
# Deliberate error
std_axis_3 = tf.math.reduce_std(E, 3)

print("tensor E: ", E)
print("\nstd_axis_0: ", std_axis_3)

InvalidArgumentError: ignored

### Find the positional maximum and minimum of a tensor (`tf.math.argmax` & `tf.math.argmin`)

In [86]:
# Create a new tensor
tf.random.set_seed(42)
F = tf.random.uniform(shape=[1000], dtype=tf.float16)
F, F.shape, F.ndim

(<tf.Tensor: shape=(1000,), dtype=float16, numpy=
 array([0.0928  , 0.7275  , 0.8135  , 0.04102 , 0.746   , 0.836   ,
        0.042   , 0.0654  , 0.0654  , 0.4287  , 0.699   , 0.375   ,
        0.06934 , 0.7393  , 0.1953  , 0.207   , 0.5264  , 0.922   ,
        0.1885  , 0.8975  , 0.4287  , 0.4766  , 0.1768  , 0.576   ,
        0.9834  , 0.2979  , 0.8135  , 0.4463  , 0.623   , 0.573   ,
        0.338   , 0.8125  , 0.208   , 0.7725  , 0.371   , 0.5244  ,
        0.1357  , 0.4746  , 0.2998  , 0.2969  , 0.75    , 0.873   ,
        0.798   , 0.8037  , 0.2734  , 0.07227 , 0.9434  , 0.542   ,
        0.534   , 0.867   , 0.7383  , 0.2031  , 0.0664  , 0.2002  ,
        0.87    , 0.7354  , 0.08887 , 0.10254 , 0.702   , 0.2598  ,
        0.251   , 0.592   , 0.1621  , 0.8516  , 0.1523  , 0.675   ,
        0.6904  , 0.991   , 0.1221  , 0.666   , 0.4932  , 0.208   ,
        0.7646  , 0.006836, 0.1387  , 0.616   , 0.6016  , 0.165   ,
        0.75    , 0.1279  , 0.1533  , 0.05664 , 0.9307  , 0.7627  

In [87]:
# Argmax — returns the index with the largest value across axes of a tensor
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=618>

In [88]:
# index the largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float16, numpy=0.998>

In [89]:
# Argmin — returns the index with the smallest value across axes of a tensor
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=187>

In [90]:
# index the smallest value position
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float16, numpy=0.001953>

In [91]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>